In [ ]:
from helper import print_frame
from moviepy.editor import concatenate_videoclips, VideoFileClip, AudioFileClip
from video import split_video
from audio import get_audio_data, get_saved_audio, get_split_times, is_increasing
from other import get_unique_filename
from music_video import get_clips, build_mv_clips
from tqdm import tqdm
import numpy as np
import os
import time

VID_DIR = os.path.join('Media', 'Videos')
VID_FILES = [os.path.join(VID_DIR, f) for f in os.listdir(VID_DIR) if f.split('.')[-1].lower() in ['mp4', 'avi', 'mkv', 'm4v']]
print(f'{len(VID_FILES)} videos found.', [f.split('\\')[-1] for f in VID_FILES])

# Audio to split clips from
AUD_DIR = os.path.join('Media', 'Audio')
AUD_FILE = os.path.join(AUD_DIR, 'galaxy rise - drums.wav')
print(AUD_FILE)

# Audio to stitch to final video
FINAL_AUDIO = os.path.join(AUD_DIR, 'greydon square - galaxy rise.wav')

EXPORT_FILENAME = 'music_video.mp4'

### TODO:
- Create tool to find best audio split points
- Clean up print_frame debug in functions once no longer being used
- Export clips to clip folder
- Create threshold visualizer

### Potentionally Useful Functions
- clip.get_frame(time)
- clip.ipython_display(width=240)

In [ ]:
# video = VideoFileClip(VID_FILES[0])

In [ ]:
# print('Frames/Second:', video.reader.fps)
# print('Frame Count:', video.reader.nframes)
# print('Video Length:', video.duration)

In [ ]:
# print_frame(video.get_frame(np.random.randint(0, video.duration)))

# Split Video In To Clips

# Audio

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
saved_data = get_saved_audio(AUD_FILE)
if saved_data:
    audio_data, CHUNK, RATE = saved_data
else:
    audio_data, CHUNK, RATE = get_audio_data(AUD_FILE)

In [ ]:
plt.figure(figsize=(14,6))
plt.plot([v for chunk in audio_data[:100] for v in chunk])
plt.show()

In [ ]:
max_data = [np.max(r) for r in audio_data]
seconds = [(CHUNK / RATE)*i for i in range(len(audio_data))]

plot_time = 40 # seconds
amp_thresh = 900000000

# Get samples above threshold amplitude if they are rising and not falling
abv_thresh = [np.max(d) > amp_thresh and is_increasing(d) for d in audio_data]
#rising = [True if np.mean(c[:512]) < np.mean(c[512:]) else False for c in audio_data]
#abv_thresh = [r and a for r, a in zip(rising, abv_thresh)]

to_idx = np.where(np.array(seconds) > plot_time)[0][0] # Convert plot time in seconds to matching index
plt.figure(figsize=(14,6))
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.plot(seconds[:to_idx], max_data[:to_idx])
plt.plot([seconds[i] for i, v in enumerate(abv_thresh[:to_idx]) if v == True], [amp_thresh for v in abv_thresh[:to_idx] if v == True], 'ro')
#plt.plot([seconds[i] for i, v in enumerate(max_data[:to_idx]) if v > amp_thresh], [v for v in max_data[:to_idx] if v > amp_thresh], 'ro')
plt.show()

In [ ]:
times = get_split_times(audio_data, RATE, amp_thresh, chunk=CHUNK)

In [ ]:
# ITERATION_CNT = 100

In [ ]:
# CHUNK = Audio.CHUNK
# Audio.open_stream()
# p = Audio.p
# wf = Audio.wf
# data = wf.readframes(CHUNK)

# fifo_long = Audio.init_fifo_from_preprocessed_data(n_data, Audio.FIFO_LONG_LEN)
# fifo_short = Audio.init_fifo_from_preprocessed_data(n_data, Audio.FIFO_SHORT_LEN)

# for sample_idx in range(len(n_data)):
#     Audio.stream.write(data)
#     data = wf.readframes(CHUNK)
#     if len(data) < CHUNK:
#         break

#     lfb = n_data[sample_idx]  # Log of the frequencies for each bucket

#     # Get rolling average & min
#     roll_delta, fifo_long, fifo_short = Audio.get_roll_delta(fifo_long, fifo_short, lfb)
#     #print(np.max(roll_delta))
#     if np.max(roll_delta) > 0.05:
#         print(np.max(roll_delta))
# #     if np.max(lfb) > .37:
# #         print(np.max(lfb))
    
#     if sample_idx > ITERATION_CNT:
#         break

# Build Music Video

In [ ]:
clip_generator = get_clips(VID_FILES, single=True)

In [ ]:
start_time = time.time()
mv_clips = build_mv_clips(times, clip_generator)
print('Time taken:', (time.time() - start_time)/60)

In [ ]:
music_video = concatenate_videoclips(mv_clips)

In [ ]:
music_audio = AudioFileClip(FINAL_AUDIO).subclip(0, music_video.duration)

In [ ]:
final_music_video = music_video.set_audio(music_audio)

In [ ]:
export_name = get_unique_filename(EXPORT_FILENAME)

In [ ]:
final_music_video.write_videofile(export_name)